In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
import joblib

In [19]:
# Chargement des données
df = pd.read_csv("dataset.csv")
print(f"Nombre de lignes dans le dataset initial: {df.shape[0]}")
print(f"Colonnes dans le dataset: {df.columns.tolist()}")

Nombre de lignes dans le dataset initial: 10002
Colonnes dans le dataset: ['Film_title', 'Release_year', 'Director', 'Cast', 'Average_rating', 'Owner_rating', 'Genres', 'Runtime', 'Countries', 'Original_language', 'Spoken_languages', 'Description', 'Studios', 'Watches', 'List_appearances', 'Likes', 'Fans', '½', '★', '★½', '★★', '★★½', '★★★', '★★★½', '★★★★', '★★★★½', '★★★★★', 'Total_ratings', 'Film_URL']


In [20]:
# Sélectionnons uniquement les colonnes essentielles pour notre modèle
essential_columns = ['Release_year', 'Director', 'Cast', 'Average_rating', 'Genres', 'Runtime']
df_model = df[essential_columns].copy()
print(f"Nombre de lignes dans df_model: {df_model.shape[0]}")

Nombre de lignes dans df_model: 10002


In [21]:
# Affichons le nombre de valeurs manquantes pour chaque colonne
print("Nombre de valeurs manquantes par colonne:")
for col in df_model.columns:
    print(f"{col}: {df_model[col].isna().sum()}")

Nombre de valeurs manquantes par colonne:
Release_year: 10002
Director: 10
Cast: 57
Average_rating: 581
Genres: 24
Runtime: 10


In [22]:
# Filtrons le dataset pour conserver uniquement les lignes où Average_rating n'est pas NaN
df_model = df_model[~df_model['Average_rating'].isna()].copy()
print(f"Nombre de lignes après avoir filtré les Average_rating manquantes: {df_model.shape[0]}")

Nombre de lignes après avoir filtré les Average_rating manquantes: 9421


In [23]:
# Vérifions à nouveau le nombre de valeurs manquantes
print("Nombre de valeurs manquantes par colonne après filtrage:")
for col in df_model.columns:
    print(f"{col}: {df_model[col].isna().sum()}")

Nombre de valeurs manquantes par colonne après filtrage:
Release_year: 9421
Director: 7
Cast: 41
Average_rating: 0
Genres: 8
Runtime: 2


In [24]:
# Vérifier si Runtime a toutes ses valeurs manquantes
if df_model['Runtime'].isna().all():
    print("Toutes les valeurs de Runtime sont manquantes, nous allons ajouter une valeur constante")
    df_model['Runtime'] = 90  # Valeur constante par défaut (90 minutes)

In [25]:
# Remplacement des valeurs manquantes pour les colonnes catégorielles
df_model['Director'] = df_model['Director'].fillna('Unknown')
df_model['Cast'] = df_model['Cast'].fillna('Unknown')
df_model['Genres'] = df_model['Genres'].fillna('Unknown')

In [26]:
# Pour les valeurs numériques, remplaçons par la médiane ou 0 si toutes les valeurs sont NaN
if not df_model['Release_year'].isna().all():
    df_model['Release_year'] = df_model['Release_year'].fillna(df_model['Release_year'].median())
if not df_model['Runtime'].isna().all():
    df_model['Runtime'] = df_model['Runtime'].fillna(df_model['Runtime'].median())

In [27]:
print(f"Nombre de lignes après traitement des valeurs manquantes: {df_model.shape[0]}")

Nombre de lignes après traitement des valeurs manquantes: 9421


In [28]:
# Affichons quelques informations sur le dataset après prétraitement
print("\nAperçu des données après prétraitement:")
print(df_model.head())
print("\nRésumé statistique des colonnes numériques:")
print(df_model.describe())


Aperçu des données après prétraitement:
   Release_year         Director  \
0           NaN  Eckhart Schmidt   
1           NaN    George Miller   
2           NaN    Dario Argento   
3           NaN    Sofia Coppola   
4           NaN  Katsuhiro Otomo   

                                                Cast  Average_rating  \
0  ['Désirée Nosbusch', 'Bodo Staiger', 'Simone B...            3.57   
1  ['Tom Hardy', 'Charlize Theron', 'Nicholas Hou...            4.18   
2  ['Jessica Harper', 'Stefania Casini', 'Flavio ...            3.93   
3  ['Bill Murray', 'Scarlett Johansson', 'Akiko T...            3.79   
4  ['Mitsuo Iwata', 'Nozomu Sasaki', 'Mami Koyama...            4.28   

                                       Genres  Runtime  
0                         ['Horror', 'Drama']     92.0  
1  ['Adventure', 'Science Fiction', 'Action']    121.0  
2                                  ['Horror']     99.0  
3              ['Drama', 'Comedy', 'Romance']    102.0  
4  ['Animation', 'Action

In [29]:
# Séparation features et target
X = df_model.drop('Average_rating', axis=1)
y = df_model['Average_rating']

In [30]:
# Définition des colonnes numériques et catégorielles
categorical_features = ['Genres', 'Cast', 'Director']
numeric_features = ['Release_year', 'Runtime']

In [31]:
# Préparation des transformateurs pour le pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

In [32]:
# Combinaison des transformateurs
preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, categorical_features),
    ('num', numeric_transformer, numeric_features)
])

In [33]:
# Définition du pipeline complet
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [34]:
# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Dimensions de X_train: {X_train.shape}")
print(f"Dimensions de X_test: {X_test.shape}")

Dimensions de X_train: (7536, 5)
Dimensions de X_test: (1885, 5)


In [35]:
# Entraînement du modèle
model.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniconda/base/envs/emotion-detection/lib/python3.10/site-packages/sklearn/impute/_base.py:572: FutureWarning: Currently, when `keep_empty_feature=False` and `strategy="constant"`, empty features are not dropped. This behaviour will change in version 1.8. Set `keep_empty_feature=True` to preserve this behaviour.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Unknown',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Genres', 'Cast',
                                                   'Director']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Release_year',
                                                   'Runtime'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [36]:
# Évaluation
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

RMSE: 0.47


In [37]:
# Sauvegarde du modèle
joblib.dump(model, "movie_rating_model.joblib")
print("Modèle sauvegardé sous 'movie_rating_model.joblib'")

Modèle sauvegardé sous 'movie_rating_model.joblib'
